<a href="https://colab.research.google.com/github/Sepren/LLM_for_business_context_creation/blob/main/LLM_for_business_context_creation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



```
# Выбран кодовый формат
```

В работе используется модель FRED-T5

# Загрузка датасета

In [ ]:
!git clone https://github.com/awe-f0x/LLM-for-business-content-creation.git
DATA_PATH = "LLM-for-business-content-creation/data/dataset.json"

import json
with open(DATA_PATH, 'r', encoding='utf-8') as f:
    data = json.load(f)

print(f"Всего текстов: {len(data)}")

fatal: destination path 'LLM-for-business-content-creation' already exists and is not an empty directory.
Всего текстов: 1040


In [ ]:
sample_key = next(iter(data))
sample_text = data[sample_key]['text']

print("=== Оригинальный текст ===")
print(sample_text[:300] + "...")
print(f"\nДлина: {len(sample_text.split())} слов")

=== Оригинальный текст ===
Прибыль сектора сократилась
до 770 млрд руб. 1 (-14% 
по сравнению с 1к24), 
в основном из-за роста операционных расходов, снижения доходов от инвалюты, а также убытка от ценных бумаг (повлияла отрицательная переоценка ОФЗ вследствие ожиданий по ужесточению ДКУ).
Рост ипотеки ускорился 
до 6,3% 2 (2...

Длина: 309 слов


# Проверка данных

In [ ]:
print(f"Тип данных: {type(data)}")

if isinstance(data, list):
    print(f"Всего текстов: {len(data)}")
    if len(data) > 0:
        print("\nПример текста:")
        print(json.dumps(data[0], ensure_ascii=False, indent=2))
elif isinstance(data, dict):
    print("\nДанные в формате словаря. Доступные ключи:")
    print(list(data.keys()))
    # Выведем первый элемент, если есть вложенные данные
    first_key = next(iter(data))
    print(f"\nПример данных по ключу '{first_key}':")
    print(json.dumps(data[first_key], ensure_ascii=False, indent=2))
else:
    print("Неизвестный формат данных.")

Тип данных: <class 'dict'>

Данные в формате словаря. Доступные ключи:
['8d278789-c952-4ed6-ad5d-65e0859edafa', 'a03c5fbe-da58-4c51-9120-76e0169c270d', 'aec2d27b-ca54-48fc-bd09-7e6f2527ed39', '308f4c83-b52c-4d3e-bc84-110ce02ee060', '08132b18-4235-4b94-be59-670bd05928a3', '55a0e018-345e-4a5e-8f37-cf79cb270e5c', '46d7df7e-7a8e-40ec-b5e8-093a5904a664', 'db336eb5-77cd-418f-b27d-0ba3bf39306e', 'a84b82f5-963f-4688-b330-6b85d690a6b5', '474ba176-b460-4997-9800-c3a8160c85a5', 'cf728d14-9731-451e-8c6c-d273a88cff27', 'b4333e8a-fc79-4ccb-9b2d-4113e95ba04f', 'f6171eb2-db0e-4027-8a95-da4819c972a7', 'aa9097b8-bd0a-459f-ae19-2eb569276717', '400ae9cf-3803-40ae-93f5-f3a0554f000b', '434ed983-6dda-4c10-841e-977d2a59f20e', '71434e9e-2589-4dad-bcb1-294c4159a153', '74e79c20-6c69-4d2a-a732-6e74b8b08138', '7950675f-2376-4003-b59a-edf9ba3ed3f7', '79eadaf2-91c3-4e30-8a87-db6b03f8f86b', '2c39ea7d-1083-455d-b968-1af5142904f4', '12e601ed-1219-4cbd-a0aa-c1c0426340f5', 'ff652931-49a5-4d2d-9991-af00b5ad61b1', '48e15be

# Установка зависимостей

In [ ]:
!pip install transformers sentencepiece pymorphy3 syllables

# Импорт библиотек и инициализация

In [ ]:
import re
import pymorphy3
import syllables
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Инициализация анализатора
morph = pymorphy3.MorphAnalyzer()

# Загрузка модели
model_name = "cointegrated/rut5-base-absum"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/315 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/828k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


config.json:   0%|          | 0.00/753 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/977M [00:00<?, ?B/s]

# Метрика AlinaEstimator

In [ ]:
class AlinaEstimator:
    @staticmethod
    def calculate(text):
        """Оценка прозрачности 0-10 на основе:
        - Лексики, морфологии, синтаксиса"""
        words = re.findall(r'\w+', text.lower())
        if not words:
            return 0.0

        # Лексическая сложность
        unique_ratio = len(set(words)) / len(words)

        # Морфологическая сложность
        long_words = sum(1 for w in words if len(w) > 8) / len(words)

        # Синтаксическая сложность
        sentences = [s for s in re.split(r'(?<=[.!?]) +', text) if s]
        avg_sent_len = sum(len(s.split()) for s in sentences) / len(sentences) if sentences else 0

        # Итоговая формула
        score = 10 * (1 - 0.4*unique_ratio - 0.4*long_words - 0.2*(avg_sent_len/20))
        return max(0, min(round(score, 1), 10))

# Метрика FRE

In [ ]:
class FRECalculator:
    @staticmethod
    def count_syllables_ru(word):
        return len(re.findall(r'[аеёиоуыэюя]', word.lower()))
    @staticmethod
    def calculate(text):
        """Реализация Flesch Reading Ease для русского языка"""
        words = re.findall(r'\w+', text)
        sentences = [s for s in re.split(r'(?<=[.!?]) +', text) if s]

        if not words or not sentences:
            return 0.0

        # Подсчет слогов
        syllable_count = sum(FRECalculator.count_syllables_ru(w) for w in words)
        fre = 206.835 - (1.3 * (len(words)/len(sentences))) - (60.1 * (syllable_count/len(words)))
        return max(0, min(round(fre, 1), 100))

# Ищем термины

In [ ]:
class TermDetector:
    def __init__(self):
        self.common_words = {
            'финансовый', 'банковский', 'кредитный',
            'регулирование', 'система', 'рынок'
        }

    def find_terms(self, text):
        words = re.findall(r'\b[а-яё]{8,}\b', text.lower())
        abbreviations = re.findall(r'\b[А-Я]{2,}\b', text)

        terms = []
        for word in set(words + abbreviations):
            parsed = morph.parse(word)[0]
            if parsed.normal_form not in self.common_words:
                terms.append(word)
        return terms

# Упрощаем текст

In [ ]:
class TextSimplifier:
    def __init__(self):
        self.term_detector = TermDetector()

    def simplify(self, text):
        terms = self.term_detector.find_terms(text)
        # print(f"Найдены термины: {terms}")

        for term in terms:
            if re.fullmatch(r'[А-Я]{2,}', term):
                text = text.replace(term, f"{term}") #(расшифровка)
            # else:
            #     root = morph.parse(term)[0].normal_form[:6] + ".."
            #     text = text.replace(term, root)

        text = self._simplify_syntax(text)
        return text

    def _simplify_syntax(self, text):
        sentences = [s for s in re.split(r'(?<=[.!?]) +', text) if s]
        simplified = []

        for sent in sentences:
            if len(sent.split()) > 15:
                parts = re.split(r'[,;] (?:и|а|но|что|которые) ', sent)
                simplified.extend(p for p in parts if p)
            else:
                simplified.append(sent)

        result = []
        for sent in simplified:
            # sent = re.sub(r'был[ао]?\s(\w+ен\w*)', r'\1или', sent)  # Пассив в актив
            # sent = re.sub(r'\bосуществляется\b', 'проводится', sent)
            result.append(sent)

        return ' '.join(result)

# LLM

In [ ]:
class TextRefiner:
    def refine(self, text):
        sentences = re.split(r'(?<=[.!?])\s+', text.strip())
        simplified_sentences = []

        for sentence in sentences:
            if not sentence.strip():
                continue

            prompt = f"Текст: {sentence.strip()} \nПереписанный вариант:"

            print('prompt:', prompt)

            inputs = tokenizer(
                prompt,
                return_tensors="pt",
                max_length=512,
                truncation=True
            )

            outputs = model.generate(
                inputs.input_ids,
                max_length=512,
                num_beams=3,
                no_repeat_ngram_size=3,
                early_stopping=True
            )

            simplified = tokenizer.decode(outputs[0], skip_special_tokens=True)
            simplified_sentences.append(simplified.strip())
            print('simplified:', simplified)

        return ' '.join(simplified_sentences)


In [ ]:
class Pipeline:
    def __init__(self):
        self.simplifier = TextSimplifier()
        self.refiner = TextRefiner()
        self.alina = AlinaEstimator()
        self.fre = FRECalculator()

    def process(self, text):
        # print("=== Исходный текст ===\n", text + "\n")

        # Оценка качества
        orig_metrics = {
            'Alina': self.alina.calculate(text),
            'FRE': self.fre.calculate(text)
        }


        simplified = self.simplifier.simplify(text)
        refined = self.refiner.refine(simplified)


        refined_metrics = {
            'Alina': self.alina.calculate(refined),
            'FRE': self.fre.calculate(refined)
        }

        return {
            'text': refined,
            'metrics': {
                'original': orig_metrics,
                'simplified': refined_metrics
            }
        }

# Пример использования

In [ ]:
pipeline = Pipeline()

alina_scores_before = []
alina_scores_after = []
fre_scores_before = []
fre_scores_after = []

# Берем первые 10 элементов
for i, (key, item) in enumerate(data.items()):
    if i >= 10:
        break

    text = item['text']
    print(f"\n================ ТЕКСТ {i + 1} ================\n")

    result = pipeline.process(text)

    # print("=== Результат ===")
    # print(result['text'])
    print("=== Метрики ===")
    orig_alina = result['metrics']['original']['Alina']
    simp_alina = result['metrics']['simplified']['Alina']
    orig_fre = result['metrics']['original']['FRE']
    simp_fre = result['metrics']['simplified']['FRE']

    print(f"Alina: {orig_alina} → {simp_alina}")
    print(f"FRE: {orig_fre} → {simp_fre}")

    alina_scores_before.append(orig_alina)
    alina_scores_after.append(simp_alina)
    fre_scores_before.append(orig_fre)
    fre_scores_after.append(simp_fre)

# Вывод средних метрик
def avg(lst):
    return round(sum(lst) / len(lst), 2) if lst else 0.0

print("\n========== СРЕДНИЕ МЕТРИКИ ==========")
print(f"Alina: {avg(alina_scores_before)} → {avg(alina_scores_after)}")
print(f"FRE: {avg(fre_scores_before)} → {avg(fre_scores_after)}")



================ ТЕКСТ 1 ================

prompt: Текст: Прибыль сектора сократилась
до 770 млрд руб. 
Переписанный вариант:
simplified: Прибыль сектора сократилась до 770 млрд руб.
prompt: Текст: 1 (-14% 
по сравнению с 1к24), 
в основном из-за роста операционных расходов, снижения доходов от инвалюты также убытка от ценных бумаг (повлияла отрицательная переоценка ОФЗ вследствие ожиданий по ужесточению ДКУ). 
Переписанный вариант:
simplified: Эксперты считают, что это отрицательная переоценка ОФЗ, а также снижение доходов от инвалюты.
prompt: Текст: Рост ипотеки ускорился 
до 6,3% 2 (2,6% в 1к24) 
в связи с ожидаемым существенным сокращением господдержки с 01.07.24. 
Переписанный вариант:
simplified: Кредитный рост ипотеки ускорился до 6,3% 2 (2,6% в 1к24).
prompt: Текст: Выдачи рыночной ипотеки оставались невысокими. 
Переписанный вариант:
simplified: Цена ипотеки была невысокой.
prompt: Текст: Потребительское кредитование продолжило ускоряться (до 5,9 с 3,7% 
в 1к24) связано с выс